In [1]:
using Pkg

Pkg.activate(".")

  Activating project at `~/Documents/Uni/Master/MA/visual_analysis`


In [2]:
using GLMakie
using GeoMakie
using EmpiricalOrthogonalFunctions
using NCDatasets 
using Dates
using BenchmarkTools
using Statistics

In [3]:
data_path = "/home/denis/workspace/data/ivt_fields_v1"

function get_files_of_member(scenario_id, member_nr)
    return readdir(joinpath(data_path, scenario_id, "r$(member_nr)i1p1f1"), join = true)    
end


get_files_of_member (generic function with 1 method)

In [4]:
function get_eof_of_datachunk(data)
    eof = EmpiricalOrthogonalFunction(data; timedim=3)

    temporalsignal = pcs(eof)
    spatialsignal = reshape(eofs(eof),(size(datain)[1:2]..., :))
    return temporalsignal, spatialsignal
end

get_eof_of_datachunk (generic function with 1 method)

In [5]:
function get_all_data(scenario_id, member_nr)

    file_paths = get_files_of_member(scenario_id, member_nr)

    time_data = Union{Missing, Dates.DateTime}[]
    ivt_data = Array{Union{Missing, Float64}, 3}[]

    

    for file_path in file_paths
        time_chunk, ivt_chunk = NCDataset(file_path) do ds
            return ds[:time][:]::Array{Union{Missing, Dates.DateTime}, 1}, ds[:ivt][:, :, :]::Array{Union{Missing, Float64}, 3}
        end
        append!(time_data, time_chunk)
        push!(ivt_data, ivt_chunk)
    end

    return time_data, cat(ivt_data..., dims=3)
end

get_all_data (generic function with 1 method)

In [7]:
time_data, ivt_data = get_all_data("ssp126", 10)

InterruptException: InterruptException:

In [6]:
time_data, ivt_data_ssp126 = NCDataset(get_files_of_member("ssp126", 10)[1]) do ds
    return ds[:time][:]::Array{Union{Missing, Dates.DateTime}, 1}, ds[:ivt][:, :, :]::Array{Union{Missing, Float64}, 3}
end

(Union{Missing, DateTime}[DateTime("2015-01-01T06:00:00"), DateTime("2015-01-01T12:00:00"), DateTime("2015-01-01T18:00:00"), DateTime("2015-01-02T00:00:00"), DateTime("2015-01-02T06:00:00"), DateTime("2015-01-02T12:00:00"), DateTime("2015-01-02T18:00:00"), DateTime("2015-01-03T00:00:00"), DateTime("2015-01-03T06:00:00"), DateTime("2015-01-03T12:00:00")  …  DateTime("2034-12-29T18:00:00"), DateTime("2034-12-30T00:00:00"), DateTime("2034-12-30T06:00:00"), DateTime("2034-12-30T12:00:00"), DateTime("2034-12-30T18:00:00"), DateTime("2034-12-31T00:00:00"), DateTime("2034-12-31T06:00:00"), DateTime("2034-12-31T12:00:00"), DateTime("2034-12-31T18:00:00"), DateTime("2035-01-01T00:00:00")], Union{Missing, Float64}[107.83024404217494 105.35183652372383 … 4.810704767513298 14.481626430503018; 138.11806601868335 132.92639636951492 … 5.865105532526301 14.261222253533777; … ; 32.42546716028079 93.37890870191428 … 7.1671847647811555 15.350266308663777; 35.96681207349521 100.77858958675463 … 6.00310975

In [7]:
ivt_data_ssp585 = NCDataset(get_files_of_member("ssp585", 10)[1]) do ds
    return ds[:ivt][:, :, :]::Array{Union{Missing, Float64}, 3}
end

70×32×29220 Array{Union{Missing, Float64}, 3}:
[:, :, 1] =
 107.779   105.269    99.1085   69.1999  …   2.09019  4.84514  14.4733
 138.146   132.656   110.339    78.5921      3.87038  5.87401  14.3014
 154.812   143.629   104.327    89.1332      5.08567  6.96693  13.5715
 156.268   139.904   100.601    83.9453      6.36022  7.89291  12.5927
 162.571   136.287    99.9174   72.6222      7.32861  8.4677   11.4227
 158.79    151.618   131.201   116.547   …   8.13568  8.44744  10.1762
 140.949   163.104   160.537   155.979       8.83942  8.15491   9.01402
 127.855   177.775   191.284   178.838      10.1558   7.60136   7.98729
 117.079   186.908   215.259   197.29       11.5956   7.24157   7.18241
 109.702   183.934   240.219   226.005      12.6642   7.36459   6.62246
   ⋮                                     ⋱            ⋮        
 112.076   139.184    90.2237   84.9278     27.0009   8.82352  12.0548
 105.661   144.456   112.11     79.9281     29.6993   8.76015  12.8203
  92.9012  143.63    

In [32]:
size(ivt_data_ssp585)

(70, 32, 29220)

In [8]:
function filter_by_date(fun, time_data, ivt_data, time_dim = 3)

    time_result = filter(fun, time_data)
    time_indices = [i for i in eachindex(time_data) if fun(time_data[i])]

    return time_result, ivt_data[:, :, time_indices]
end

filter_by_date (generic function with 2 methods)

In [9]:
winter_months = [12, 1, 2, 3]

4-element Vector{Int64}:
 12
  1
  2
  3

In [11]:
time_filtered, data_filtered_ssp126 = filter_by_date(time_data, ivt_data_ssp126) do time_element
    

    for wm in winter_months
        if month(time_element) == wm
            return true
        end
    end
    return false
end

(Union{Missing, DateTime}[DateTime("2015-01-01T06:00:00"), DateTime("2015-01-01T12:00:00"), DateTime("2015-01-01T18:00:00"), DateTime("2015-01-02T00:00:00"), DateTime("2015-01-02T06:00:00"), DateTime("2015-01-02T12:00:00"), DateTime("2015-01-02T18:00:00"), DateTime("2015-01-03T00:00:00"), DateTime("2015-01-03T06:00:00"), DateTime("2015-01-03T12:00:00")  …  DateTime("2034-12-29T18:00:00"), DateTime("2034-12-30T00:00:00"), DateTime("2034-12-30T06:00:00"), DateTime("2034-12-30T12:00:00"), DateTime("2034-12-30T18:00:00"), DateTime("2034-12-31T00:00:00"), DateTime("2034-12-31T06:00:00"), DateTime("2034-12-31T12:00:00"), DateTime("2034-12-31T18:00:00"), DateTime("2035-01-01T00:00:00")], Union{Missing, Float64}[107.83024404217494 105.35183652372383 … 4.810704767513298 14.481626430503018; 138.11806601868335 132.92639636951492 … 5.865105532526301 14.261222253533777; … ; 32.42546716028079 93.37890870191428 … 7.1671847647811555 15.350266308663777; 35.96681207349521 100.77858958675463 … 6.00310975

In [12]:
_, data_filtered_ssp585 = filter_by_date(time_data, ivt_data_ssp585) do time_element
    

    for wm in winter_months
        if month(time_element) == wm
            return true
        end
    end
    return false
end

(Union{Missing, DateTime}[DateTime("2015-01-01T06:00:00"), DateTime("2015-01-01T12:00:00"), DateTime("2015-01-01T18:00:00"), DateTime("2015-01-02T00:00:00"), DateTime("2015-01-02T06:00:00"), DateTime("2015-01-02T12:00:00"), DateTime("2015-01-02T18:00:00"), DateTime("2015-01-03T00:00:00"), DateTime("2015-01-03T06:00:00"), DateTime("2015-01-03T12:00:00")  …  DateTime("2034-12-29T18:00:00"), DateTime("2034-12-30T00:00:00"), DateTime("2034-12-30T06:00:00"), DateTime("2034-12-30T12:00:00"), DateTime("2034-12-30T18:00:00"), DateTime("2034-12-31T00:00:00"), DateTime("2034-12-31T06:00:00"), DateTime("2034-12-31T12:00:00"), DateTime("2034-12-31T18:00:00"), DateTime("2035-01-01T00:00:00")], Union{Missing, Float64}[107.778753336393 105.2688578161348 … 4.84513696356521 14.473312588426843; 138.1462566198071 132.65618873256656 … 5.874014466553617 14.301361694762948; … ; 32.48572331123109 93.58189447496002 … 7.1780321470992545 15.377072390249369; 35.949182269846716 100.84770349490273 … 6.003913533728

In [21]:
years = Dict()

for d in time_filtered
    key = year(d)
    if haskey(years, key)
        push!(years[key], d)
    else
        years[key] = [d]
    end
end 
years[2017]

484-element Vector{DateTime}:
 2017-01-01T00:00:00
 2017-01-01T06:00:00
 2017-01-01T12:00:00
 2017-01-01T18:00:00
 2017-01-02T00:00:00
 2017-01-02T06:00:00
 2017-01-02T12:00:00
 2017-01-02T18:00:00
 2017-01-03T00:00:00
 2017-01-03T06:00:00
 ⋮
 2017-12-29T18:00:00
 2017-12-30T00:00:00
 2017-12-30T06:00:00
 2017-12-30T12:00:00
 2017-12-30T18:00:00
 2017-12-31T00:00:00
 2017-12-31T06:00:00
 2017-12-31T12:00:00
 2017-12-31T18:00:00

In [14]:
maximum(data_filtered)

1927.2673016598856

In [13]:
lons, lats = NCDataset(get_files_of_member("ssp126", 10)[1]) do ds
    return ds[:lon][:]::Array{Union{Missing, Float64}, 1}, ds[:lat][:]::Array{Union{Missing, Float64}, 1}
end

(Union{Missing, Float64}[-90.0, -88.125, -86.25, -84.375, -82.5, -80.625, -78.75, -76.875, -75.0, -73.125  …  22.5, 24.375, 26.25, 28.125, 30.0, 31.875, 33.75, 35.625, 37.5, 39.375], Union{Missing, Float64}[21.450475037398185, 23.31573072614093, 25.180985581270594, 27.04623949994481, 28.91149236871774, 30.77674406172325, 32.64199443851768, 34.50724334150103, 36.37249059281224, 38.23773599056483  …  62.48557052203639, 64.35073040887207, 66.2158721139987, 68.08099098565125, 69.94608064698343, 71.81113211427447, 73.67613231320912, 75.54106145287895, 77.4058880820788, 79.27055903485967])

In [32]:
findfirst(x -> x > 58, lats)

21

In [33]:
data_filtered[24, 22, 1]

9.42627982291488

In [14]:
struct ScenarioData
    name::String
    data::AbstractArray{Union{Missing, <: AbstractFloat}, 3}
end

struct TimelineData
    lons::Vector{Union{Missing, <: AbstractFloat}}
    lats::Vector{Union{Missing, <: AbstractFloat}}
    time::Vector{Union{Missing, Dates.DateTime}}
    datasets::Vector{ScenarioData}
end

In [27]:

# create geographical axes
function local_geoaxis_creation!(
    figure::Makie.Figure,
    lonlims::Tuple{<:Number, <:Number}, 
    latlims::Tuple{<:Number, <:Number}; 
    lonpadding::Float64 = 1.0, 
    latpadding::Float64 = 1.0,
    figure_row::Int = 1,
    figure_col::Int = 1,
    title = ""
    )
    
    paddingfun = (x, y) -> x < 0 ? x-y : x + y

    lon_padded = paddingfun.(lonlims, lonpadding)
    lon_center = lon_padded[1] + ((lon_padded[2] - lon_padded[1]) / 2)



    lat_padded = paddingfun.(latlims, latpadding)
    lat_center = lat_padded[1] + ((lat_padded[2] - lat_padded[1]) / 2)
    
    geoaxis = GeoMakie.GeoAxis(
        figure[figure_row, figure_col]; 
        # dest = "+proj=merc +lon_0=$(lon_center) +lat_0=$(lat_center)",
        dest = "+proj=merc",
        # source = dest,
        # lonlims = lonlims,
        # latlims = latlims,
        limits=(lonlims, latlims),
        title = title
    )

    

    return geoaxis
end


function build_figure(
    data,
    lon_bounds::Tuple{<:Number, <:Number},
    lat_bounds::Tuple{<:Number, <:Number};
    title::String = "",
    lonpadding::Float64 = 1.0, 
    latpadding::Float64 = 1.0,
    colormap = :viridis,
    colorrange::Union{Nothing, Tuple{<:Real, <:Real}} = nothing, 
    shading = Makie.automatic,
    resolution::Union{Nothing, Tuple{Int, Int}} = nothing
    )::Makie.Figure

    fig = isnothing(resolution) ?  Figure(fontsize=12) : Figure(size = resolution, fontsize=12)

    

    ga = local_geoaxis_creation!(fig, lon_bounds, lat_bounds; lonpadding, latpadding, title=title)
    surface!(ga, lon_bounds[1]..lon_bounds[2], lat_bounds[1]..lat_bounds[2], data; shading = shading, colormap = colormap)
    lines!(ga, GeoMakie.coastlines(); color = :white, transformation = (; translation = (0, 0, 1000)))
    return fig
end

build_figure (generic function with 1 method)

In [26]:
build_figure(data_filtered[:, :, 1], (-90, 40), (20, 80); title = "Test IVT 1", colormap = :managua100)

In [36]:

function animate_timeline(
    data::TimelineData,
    filename::String;
    framerate::Int = 30,
    lonpadding::Float64 = 1.0, 
    latpadding::Float64 = 1.0,
    colormap = :viridis,
    coastline_color = :white,
    colorrange::Union{Nothing, Tuple{<:Real, <:Real}} = nothing, 
    shading = Makie.automatic,
    resolution::Union{Nothing, Tuple{Int, Int}} = nothing
    )::Makie.Figure

    fig = isnothing(resolution) ?  Figure(fontsize=12) : Figure(size = resolution, fontsize=12)

    time_index = Observable(1)

    axis = Dict()

    lon_bounds = extrema(data.lons)
    lat_bounds = extrema(data.lats)

    all_extrema = extrema.([d.data for d in data.datasets])

    min_val, max_val = reduce((a, b) -> (min(a[1], b[1]), max(a[2], b[2])), all_extrema)
    Colorbar(fig[2, 1:2], limits = (min_val, max_val), colormap = colormap, vertical = false,  label = "IVT (norm) in kg s^-1 m^-1")

    for (i, dataset) in enumerate(data.datasets)
        axis[dataset.name] = local_geoaxis_creation!(fig, lon_bounds, lat_bounds; lonpadding, latpadding, title=@lift("$(dataset.name) $(Dates.format(data.time[$time_index], "mm/yyyy"))"), figure_col=i)
    end
    
    for dataset in data.datasets

        array_slice = @lift(dataset.data[:, :, $time_index])
        
        surface!(axis[dataset.name], lon_bounds[1]..lon_bounds[2], lat_bounds[1]..lat_bounds[2], array_slice; shading = shading, colormap = colormap, colorrange = (min_val, max_val))
        lines!(axis[dataset.name], GeoMakie.coastlines(); color = coastline_color, transformation = (; translation = (0, 0, 1000)))
    end

    timestamps = eachindex(data.time)
    record(fig, filename, timestamps; framerate = framerate) do t
        time_index[] = t
    end

    
    return fig
end

animate_timeline (generic function with 1 method)

In [23]:
tline_data = TimelineData(
    lons,
    lats,
    time_filtered,
    [
        ScenarioData(
            "ssp126",
            data_filtered_ssp126
        ),
        ScenarioData(
            "ssp585",
            data_filtered_ssp585
        )
    ]
)

TimelineData(Union{Missing, AbstractFloat}[-90.0, -88.125, -86.25, -84.375, -82.5, -80.625, -78.75, -76.875, -75.0, -73.125  …  22.5, 24.375, 26.25, 28.125, 30.0, 31.875, 33.75, 35.625, 37.5, 39.375], Union{Missing, AbstractFloat}[21.450475037398185, 23.31573072614093, 25.180985581270594, 27.04623949994481, 28.91149236871774, 30.77674406172325, 32.64199443851768, 34.50724334150103, 36.37249059281224, 38.23773599056483  …  62.48557052203639, 64.35073040887207, 66.2158721139987, 68.08099098565125, 69.94608064698343, 71.81113211427447, 73.67613231320912, 75.54106145287895, 77.4058880820788, 79.27055903485967], Union{Missing, DateTime}[DateTime("2015-01-01T06:00:00"), DateTime("2015-01-01T12:00:00"), DateTime("2015-01-01T18:00:00"), DateTime("2015-01-02T00:00:00"), DateTime("2015-01-02T06:00:00"), DateTime("2015-01-02T12:00:00"), DateTime("2015-01-02T18:00:00"), DateTime("2015-01-03T00:00:00"), DateTime("2015-01-03T06:00:00"), DateTime("2015-01-03T12:00:00")  …  DateTime("2034-12-29T18:00:

In [37]:
# animate_timeline(tline_data, "scenario_comparison_2.mp4"; colormap = :managua100, resolution = (1000, 1000))
animate_timeline(tline_data, "scenario_comparison.mp4"; colormap = :managua100)